<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkDataFrame/SparkDataFrameNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [2]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/utils.py  > /dev/null 2>&1

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

**Spark UI section**
<br>To use Spark UI, uncomment below sections and use the public link

In [4]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [5]:
#!curl -s http://localhost:4040/api/tunnels

**Main Section**

In [6]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .enableHiveSupport() \
        .getOrCreate()

logger = Log4J(spark)

**Basic CSV Read**

In [7]:
def load_csv_file(spark, url):
   spark.sparkContext.addFile(url)

   survey_df=spark.read \
   .format("csv") \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .option("mode", "FAILFAST") \
   .load('file://'+SparkFiles.get("sample.csv"))

   return survey_df

def count_by_country(survey_df):
  count_df= survey_df.select("Age", "Gender", "Country", "State") \
                     .where("Age <= 40") \
                     .groupBy("Country") \
                     .count()
  return count_df

In [8]:
logger.info("Start CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/sample.csv'

survey_df=load_csv_file(spark, url)
partitioned_survey_df=survey_df.repartition(2)

count_df=count_by_country(partitioned_survey_df)

logger.info(count_df.collect())

logger.info("End CSV Load")

**Data Types**<br>
(https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.types)

**CSV Read (CSV)**

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

logger.info("Start Flight Time CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.csv'

spark.sparkContext.addFile(url)

flightSchemaStruct = StructType([StructField("FL_DATE", DateType(), True),
                                 StructField("OP_CARRIER", StringType(), True),
                                 StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
                                 StructField("ORIGIN", StringType(), True),
                                 StructField("ORIGIN_CITY_NAME", StringType(), True),
                                 StructField("DEST", StringType(), True),
                                 StructField("DEST_CITY_NAME", StringType(), True),
                                 StructField("CRS_DEP_TIME", IntegerType(), True),
                                 StructField("DEP_TIME", IntegerType(), True),
                                 StructField("WHEELS_ON", IntegerType(), True),
                                 StructField("TAXI_IN", IntegerType(), True),
                                 StructField("CRS_ARR_TIME", IntegerType(), True),
                                 StructField("ARR_TIME", IntegerType(), True),
                                 StructField("CANCELLED", IntegerType(), True),
                                 StructField("DISTANCE", StringType(), True),
                                ])

flight_time_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaStruct) \
                    .load('file://'+SparkFiles.get("flight-time.csv"))

flight_time_df.show()

logger.info("End CSV Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**JSON Read**

In [10]:
logger.info("Start Flight Time JSON Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.json'

spark.sparkContext.addFile(url)

flightSchemaDDL = """FL_DATE DATE, 
                     OP_CARRIER STRING, 
                     OP_CARRIER_FL_NUM INT,
                     ORIGIN STRING,
                     ORIGIN_CITY_NAME STRING,
                     DEST STRING,
                     DEST_CITY_NAME STRING,
                     CRS_DEP_TIME INT,
                     DEP_TIME INT,
                     WHEELS_ON INT,
                     TAXI_IN INT,
                     CRS_ARR_TIME INT,
                     ARR_TIME INT,
                     CANCELLED INT,
                     DISTANCE STRING"""

flight_time_df=spark.read \
                    .format("json") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaDDL) \
                    .load('file://'+SparkFiles.get("flight-time.json"))

flight_time_df.show()

logger.info("End JSON Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Parquet Read**

In [11]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

flight_time_df.show()

logger.info("End Parquet Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Write Avro file**<br>
Added <br>
spark.jars.packages                org.apache.spark:spark-avro_2.12:3.0.0
<br>to<br>
conf/spark-defaults.conf

In [12]:
from pyspark.sql.functions  import spark_partition_id

logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("End Parquet Load")
logger.info("Number of partitions: " + str(flight_time_df.rdd.getNumPartitions()))
#Even though there are two partitions, since the records are less, only one partition is used. Can repartition to get more partitions in o/p
logger.info("Records per partition: " + str(flight_time_df.groupBy(spark_partition_id()).count().collect()))

logger.info("Start Avro write")

flight_time_df.write \
              .format("avro") \
              .mode("overwrite") \
              .option("path", "dataSink/avro/") \
              .save()

logger.info("End Avro write")

In [13]:
!ls -l dataSink/avro/

total 13088
-rw-r--r-- 1 root root 13400431 Apr 13 06:02 part-00000-3ecfe627-485b-4fef-9771-aa4a2fb059bd-c000.avro
-rw-r--r-- 1 root root        0 Apr 13 06:02 _SUCCESS


**Partioned Write**

In [14]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("Start Partitioned JSON write")

#To check file split at 10K records, look for this example
#!wc -l dataSink/json/OP_CARRIER\=DL/ORIGIN\=ATL/*
flight_time_df.write \
              .format("json") \
              .mode("overwrite") \
              .option("path", "dataSink/json/") \
              .partitionBy("OP_CARRIER", "ORIGIN") \
              .option("maxRecordsPerFile", 10000) \
              .save()

logger.info("End Partitioned JSON write")

In [15]:
!ls -l dataSink/json/

total 40
drwxr-xr-x  99 root root 4096 Apr 13 06:02 'OP_CARRIER=AA'
drwxr-xr-x  36 root root 4096 Apr 13 06:02 'OP_CARRIER=AS'
drwxr-xr-x  84 root root 4096 Apr 13 06:02 'OP_CARRIER=CO'
drwxr-xr-x 118 root root 4096 Apr 13 06:02 'OP_CARRIER=DL'
drwxr-xr-x  53 root root 4096 Apr 13 06:02 'OP_CARRIER=HP'
drwxr-xr-x 119 root root 4096 Apr 13 06:02 'OP_CARRIER=NW'
drwxr-xr-x  81 root root 4096 Apr 13 06:02 'OP_CARRIER=TW'
drwxr-xr-x 106 root root 4096 Apr 13 06:02 'OP_CARRIER=UA'
drwxr-xr-x  90 root root 4096 Apr 13 06:02 'OP_CARRIER=US'
drwxr-xr-x  58 root root 4096 Apr 13 06:03 'OP_CARRIER=WN'
-rw-r--r--   1 root root    0 Apr 13 06:03  _SUCCESS


**Managed tables**<br>
Config has been setup to write tables into *warehouse_location* folder

In [16]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("Create database")
spark.sql("CREATE DATABASE IF NOT EXISTS AIRLINES_DB")
spark.catalog.setCurrentDatabase("AIRLINES_DB")

logger.info("Write table")
flight_time_df.write \
              .mode("overwrite") \
              .bucketBy(5,"OP_CARRIER", "ORIGIN") \
              .sortBy("OP_CARRIER", "ORIGIN") \
              .saveAsTable("flight_data_tbl")

spark.sql("SELECT * FROM AIRLINES_DB.flight_data_tbl").show(10)

logger.info(spark.catalog.listTables("AIRLINES_DB"))

+----------+----------+-----------------+------+----------------+----+--------------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|      DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        AA|              369|   CMH|    Columbus, OH| DFW|Dallas/Fort Worth...|         800|     752|      920|     14|         949|     934|        0|     927|
|2000-01-01|        AA|             1171|   CMH|    Columbus, OH| DFW|Dallas/Fort Worth...|        1850|    1843|     2025|      8|        2045|    2033|        0|     927|
|2000-01-01|        AA|             1807|   CMH|    Columbus, OH| DFW|Dallas/Fort Worth...|        1300|    1258|     1429|      9|    

In [17]:
!ls -lR spark-warehouse/

spark-warehouse/:
total 4
drwxr-xr-x 3 root root 4096 Apr 13 06:03 airlines_db.db

spark-warehouse/airlines_db.db:
total 4
drwxr-xr-x 2 root root 4096 Apr 13 06:03 flight_data_tbl

spark-warehouse/airlines_db.db/flight_data_tbl:
total 3844
-rw-r--r-- 1 root root 606771 Apr 13 06:03 part-00000-9d246f0e-3b2a-404d-87bf-7397e6105419_00000.c000.snappy.parquet
-rw-r--r-- 1 root root 889584 Apr 13 06:03 part-00000-9d246f0e-3b2a-404d-87bf-7397e6105419_00001.c000.snappy.parquet
-rw-r--r-- 1 root root 894879 Apr 13 06:03 part-00000-9d246f0e-3b2a-404d-87bf-7397e6105419_00002.c000.snappy.parquet
-rw-r--r-- 1 root root 867652 Apr 13 06:03 part-00000-9d246f0e-3b2a-404d-87bf-7397e6105419_00003.c000.snappy.parquet
-rw-r--r-- 1 root root 666858 Apr 13 06:03 part-00000-9d246f0e-3b2a-404d-87bf-7397e6105419_00004.c000.snappy.parquet
-rw-r--r-- 1 root root      0 Apr 13 06:03 _SUCCESS


**Log File/Raw Data/Unstructured Data handling**

In [18]:
from pyspark.sql.functions import *

logger.info("Start Log file load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/apache_logs.txt'
spark.sparkContext.addFile(url)

raw_df=spark.read \
                    .format("text") \
                    .load('file://'+SparkFiles.get("apache_logs.txt"))

raw_df.printSchema()

log_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

logs_df = raw_df.select(regexp_extract('value', log_reg, 1).alias('ip'),
                        regexp_extract('value', log_reg, 4).alias('date'),
                        regexp_extract('value', log_reg, 6).alias('request'),
                        regexp_extract('value', log_reg, 10).alias('referrer'))
logs_df.printSchema()

logs_df.withColumn('referrer', substring_index('referrer', '/',3)) \
       .where("trim(referrer) != '-' ") \
       .groupBy('referrer') \
       .count() \
       .show(100, truncate=False)

root
 |-- value: string (nullable = true)

root
 |-- ip: string (nullable = true)
 |-- date: string (nullable = true)
 |-- request: string (nullable = true)
 |-- referrer: string (nullable = true)

+-----------------------------------+-----+
|referrer                           |count|
+-----------------------------------+-----+
|http://www.semicomplete.com        |3038 |
|http://www.google.com              |123  |
|http://www.haskell.org             |1    |
|http://www.google.de               |18   |
|http://www.google.com.kh           |1    |
|https://www.google.at              |4    |
|http://www.google.fi               |4    |
|https://www.google.co.in           |13   |
|http://unix.stackexchange.com      |8    |
|http://www.google.com.tr           |1    |
|https://www.google.ru              |5    |
|http://images.google.com           |1    |
|http://avtoads.net                 |3    |
|http://www.google.co.in            |12   |
|https://www.google.no              |2    |
|http://t.

**Handcrafted DataFrame creating Rows**

In [19]:
from pyspark.sql.types import *

my_schema=StructType([StructField("ID", StringType(), True),
                      StructField("EventDate", StringType(), True)])

my_rows=[Row("123", "21/03/2019"), 
         Row("234", "21/05/2029"), 
         Row("345", "01/02/2020"),
         Row("456", "09/12/2018"),
         Row("567", "03/06/2019"),]

my_rdd=spark.sparkContext.parallelize(my_rows, 2)

my_df=spark.createDataFrame(my_rdd, my_schema)

my_df.printSchema()
my_df.show()

new_df=my_df.withColumn('EventDate', to_date('EventDate', 'd/M/y'))

new_df.printSchema()
new_df.show()

root
 |-- ID: string (nullable = true)
 |-- EventDate: string (nullable = true)

+---+----------+
| ID| EventDate|
+---+----------+
|123|21/03/2019|
|234|21/05/2029|
|345|01/02/2020|
|456|09/12/2018|
|567|03/06/2019|
+---+----------+

root
 |-- ID: string (nullable = true)
 |-- EventDate: date (nullable = true)

+---+----------+
| ID| EventDate|
+---+----------+
|123|2019-03-21|
|234|2029-05-21|
|345|2020-02-01|
|456|2018-12-09|
|567|2019-06-03|
+---+----------+



**Column Expressions**

In [20]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType
from pyspark.sql.functions import *

logger.info("Start CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/databricks-airlines.csv'

spark.sparkContext.addFile(url)

airlines_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("inferSchema", "true") \
                    .option("samplingRatio", "0.5") \
                    .load('file://'+SparkFiles.get("databricks-airlines.csv"))

#shows three ways to refer to columns
airlines_df.select("Origin", col("Dest"), airlines_df.IsArrDelayed).show(10)

+------+----+------------+
|Origin|Dest|IsArrDelayed|
+------+----+------------+
|   SAN| SFO|         YES|
|   SAN| SFO|         YES|
|   SAN| SFO|         YES|
|   SAN| SFO|          NO|
|   SAN| SFO|         YES|
|   SAN| SFO|          NO|
|   SAN| SFO|         YES|
|   SAN| SFO|         YES|
|   SAN| SFO|         YES|
|   SAN| SFO|         YES|
+------+----+------------+
only showing top 10 rows



In [21]:
airlines_df.select("Origin", "Dest", "Distance", "Year", "Month", "DayOfMonth").show(10)

airlines_df.select("Origin", "Dest", "Distance", expr("to_date(concat(Year, Month, DayOfMonth), 'yyyyMMdd') as FlightDate")).show(10)

airlines_df.select("Origin", "Dest", "Distance", to_date(concat("Year", "Month", "DayOfMonth"), 'yyyyMMdd').alias("FlightDate")).show(10)

+------+----+--------+----+-----+----------+
|Origin|Dest|Distance|Year|Month|DayOfMonth|
+------+----+--------+----+-----+----------+
|   SAN| SFO|     447|1987|   10|        14|
|   SAN| SFO|     447|1987|   10|        15|
|   SAN| SFO|     447|1987|   10|        17|
|   SAN| SFO|     447|1987|   10|        18|
|   SAN| SFO|     447|1987|   10|        19|
|   SAN| SFO|     447|1987|   10|        21|
|   SAN| SFO|     447|1987|   10|        22|
|   SAN| SFO|     447|1987|   10|        23|
|   SAN| SFO|     447|1987|   10|        24|
|   SAN| SFO|     447|1987|   10|        25|
+------+----+--------+----+-----+----------+
only showing top 10 rows

+------+----+--------+----------+
|Origin|Dest|Distance|FlightDate|
+------+----+--------+----------+
|   SAN| SFO|     447|1987-10-14|
|   SAN| SFO|     447|1987-10-15|
|   SAN| SFO|     447|1987-10-17|
|   SAN| SFO|     447|1987-10-18|
|   SAN| SFO|     447|1987-10-19|
|   SAN| SFO|     447|1987-10-21|
|   SAN| SFO|     447|1987-10-22|
|   

**User Defined Function (UDF)**<br>
to be used as column expression

In [22]:
import re
def parse_gender(gender):
  female_pattern = r'^f$|f.m|w.m'
  male_pattern = r'^m$|ma|m.l'

  if re.search(female_pattern, gender.lower()):
    return "Female"
  elif re.search(male_pattern, gender.lower()):
    return "Male"
  else:
    return "Unknown"

In [23]:
url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/survey.csv'

spark.sparkContext.addFile(url)

survey_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("survey.csv"))

survey_df.show(10)

#Column object UDF (no entry in catalog)
parse_gender_udf=udf(parse_gender, StringType())
survey_df2=survey_df.withColumn("Gender", parse_gender_udf("Gender"))
survey_df2.show(10)

#SQL UDF (entry goes into catalog)
spark.udf.register("parse_gender_udf", parse_gender, StringType())
survey_df3=survey_df.withColumn("Gender", expr("parse_gender_udf(Gender)"))
survey_df3.show(10)

spark.catalog.listFunctions()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--

[Function(name='!', description=None, className='org.apache.spark.sql.catalyst.expressions.Not', isTemporary=True),
 Function(name='%', description=None, className='org.apache.spark.sql.catalyst.expressions.Remainder', isTemporary=True),
 Function(name='&', description=None, className='org.apache.spark.sql.catalyst.expressions.BitwiseAnd', isTemporary=True),
 Function(name='*', description=None, className='org.apache.spark.sql.catalyst.expressions.Multiply', isTemporary=True),
 Function(name='+', description=None, className='org.apache.spark.sql.catalyst.expressions.Add', isTemporary=True),
 Function(name='-', description=None, className='org.apache.spark.sql.catalyst.expressions.Subtract', isTemporary=True),
 Function(name='/', description=None, className='org.apache.spark.sql.catalyst.expressions.Divide', isTemporary=True),
 Function(name='<', description=None, className='org.apache.spark.sql.catalyst.expressions.LessThan', isTemporary=True),
 Function(name='<=', description=None, cl

**Miscellaneous functions**

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, when, expr
from pyspark.sql.types import *

data_list = [("Ravi", "28", "1", "2002"),
             ("Abdul", "23", "5", "81"),  # 1981
             ("John", "12", "12", "6"),  # 2006
             ("Rosy", "7", "8", "63"),  # 1963
             ("Abdul", "23", "5", "81")]  # 1981

raw_df = spark.createDataFrame(data_list).toDF("name", "day", "month", "year")
raw_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)



In [25]:
#Add a new column

df1 = raw_df.withColumn("id", monotonically_increasing_id())
df1.show(10)

+-----+---+-----+----+----------+
| name|day|month|year|        id|
+-----+---+-----+----+----------+
| Ravi| 28|    1|2002|         0|
|Abdul| 23|    5|  81|         1|
| John| 12|   12|   6|8589934592|
| Rosy|  7|    8|  63|8589934593|
|Abdul| 23|    5|  81|8589934594|
+-----+---+-----+----+----------+



In [26]:
# CASE WHEN and CAST
df2=df1.withColumn("year", expr("""
                                CASE WHEN year < 21  THEN cast(year as int) + 2000
                                     WHEN year < 100 THEN cast(year as int) + 1900
                                     ELSE cast(year as int)
                                END
                                """))
df2.show()

df2=df1.withColumn("year", expr("""
                                CASE WHEN year < 21  THEN year + 2000
                                     WHEN year < 100 THEN year + 1900
                                     ELSE year
                                END
                                """).cast(IntegerType()))
df2.show()


+-----+---+-----+----+----------+
| name|day|month|year|        id|
+-----+---+-----+----+----------+
| Ravi| 28|    1|2002|         0|
|Abdul| 23|    5|1981|         1|
| John| 12|   12|2006|8589934592|
| Rosy|  7|    8|1963|8589934593|
|Abdul| 23|    5|1981|8589934594|
+-----+---+-----+----+----------+

+-----+---+-----+----+----------+
| name|day|month|year|        id|
+-----+---+-----+----+----------+
| Ravi| 28|    1|2002|         0|
|Abdul| 23|    5|1981|         1|
| John| 12|   12|2006|8589934592|
| Rosy|  7|    8|1963|8589934593|
|Abdul| 23|    5|1981|8589934594|
+-----+---+-----+----+----------+



In [27]:
# Cast the fields and alternative WHEN
df3=df1.withColumn("day", col("day").cast(IntegerType())) \
       .withColumn("month", col("month").cast(IntegerType())) \
       .withColumn("year", col("year").cast(IntegerType())) \

df3.printSchema()

df4=df3.withColumn("year", \
                          when(col("year") < 21, col("year") + 2000) \
                        .when(col("year") < 100, col("year") + 1900) \
                        .otherwise(col("year")))
df4.show()

root
 |-- name: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- id: long (nullable = false)

+-----+---+-----+----+----------+
| name|day|month|year|        id|
+-----+---+-----+----+----------+
| Ravi| 28|    1|2002|         0|
|Abdul| 23|    5|1981|         1|
| John| 12|   12|2006|8589934592|
| Rosy|  7|    8|1963|8589934593|
|Abdul| 23|    5|1981|8589934594|
+-----+---+-----+----+----------+



**Add/Remove columns**

In [28]:
df5=df2.withColumn("dob", expr("to_date(concat(day, '/', month, '/', year), 'd/M/y')")) \
       .drop("day", "month", "year") \
       .dropDuplicates(["name", "dob"])

df5.sort(df5.dob.desc()).show()

+-----+----------+----------+
| name|        id|       dob|
+-----+----------+----------+
| John|8589934592|2006-12-12|
| Ravi|         0|2002-01-28|
|Abdul|         1|1981-05-23|
| Rosy|8589934593|1963-08-07|
+-----+----------+----------+



**Aggregation**

In [29]:
from pyspark.sql import functions as f
url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/invoices.csv'

spark.sparkContext.addFile(url)

invoice_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("invoices.csv"))

invoice_df.show(10)

invoice_df.select(f.count("*").alias("Count *"),
                  f.sum("Quantity").alias("TotalQuantity"),
                  f.avg("UnitPrice").alias("AvgPrice"),
                  f.countDistinct("InvoiceNo").alias("CountDistinct")
                  ).show()

invoice_df.selectExpr(
                  "count(1) as `count 1`",
                  "count(StockCode) as `count field`",
                  "sum(Quantity) as TotalQuantity",
                  "avg(UnitPrice) as AvgPrice"
                ).show()

invoice_df.createOrReplaceTempView("sales")
summary_sql = spark.sql("""
      SELECT Country, InvoiceNo,
            sum(Quantity) as TotalQuantity,
            round(sum(Quantity*UnitPrice),2) as InvoiceValue
      FROM sales
      GROUP BY Country, InvoiceNo""")
summary_sql.show()

summary_df = invoice_df \
    .groupBy("Country", "InvoiceNo") \
    .agg(f.sum("Quantity").alias("TotalQuantity"),
         f.round(f.sum(f.expr("Quantity * UnitPrice")), 2).alias("InvoiceValue"),
         f.expr("round(sum(Quantity * UnitPrice),2) as InvoiceValueExpr")
         )
summary_df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     null|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

**Group By**

In [30]:
from pyspark.sql import functions as f
url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/invoices.csv'

spark.sparkContext.addFile(url)

invoice_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("invoices.csv"))

invoice_df.show(10)

NumInvoices = f.countDistinct("InvoiceNo").alias("NumInvoices")
TotalQuantity = f.sum("Quantity").alias("TotalQuantity")
InvoiceValue = f.expr("round(sum(Quantity * UnitPrice),2) as InvoiceValue")

exSummary_df = invoice_df \
    .withColumn("InvoiceDate", f.to_date(f.col("InvoiceDate"), "dd-MM-yyyy H.mm")) \
    .where("year(InvoiceDate) == 2010") \
    .withColumn("WeekNumber", f.weekofyear(f.col("InvoiceDate"))) \
    .groupBy("Country", "WeekNumber") \
    .agg(NumInvoices, TotalQuantity, InvoiceValue)

exSummary_df.sort("Country", "WeekNumber").show()

exSummary_df.coalesce(1) \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save("output")

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     null|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

In [31]:
!ls -l output/

total 4
-rw-r--r-- 1 root root 2499 Apr 13 06:03 part-00000-371fba1a-007f-4447-bbfe-44310fbac52c-c000.snappy.parquet
-rw-r--r-- 1 root root    0 Apr 13 06:03 _SUCCESS


**Windowing**

In [32]:
from pyspark.sql import functions as f
url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/summary.parquet'

spark.sparkContext.addFile(url)

summary_df=spark.read \
                    .format("parquet") \
                    .load('file://'+SparkFiles.get("summary.parquet"))

summary_df.sort("Country", "WeekNumber").show()

running_total_window = Window.partitionBy("Country") \
    .orderBy("WeekNumber") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

summary_df.withColumn("RunningTotal",
                      f.sum("InvoiceValue").over(running_total_window)) \
    .sort("Country", "WeekNumber").show()

+---------------+----------+-----------+-------------+------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|
+---------------+----------+-----------+-------------+------------+
|      Australia|        48|          1|          107|      358.25|
|      Australia|        49|          1|          214|       258.9|
|      Australia|        50|          2|          133|      387.95|
|        Austria|        50|          2|            3|      257.04|
|        Bahrain|        51|          1|           54|      205.74|
|        Belgium|        48|          1|          528|       346.1|
|        Belgium|        50|          2|          285|      625.16|
|        Belgium|        51|          2|          942|      838.65|
|Channel Islands|        49|          1|           80|      363.53|
|         Cyprus|        50|          1|          917|     1590.82|
|        Denmark|        49|          1|          454|      1281.5|
|           EIRE|        48|          7|        

**Ranking**

In [33]:
from pyspark.sql import functions as f
url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/summary.parquet'

spark.sparkContext.addFile(url)

summary_df=spark.read \
                    .format("parquet") \
                    .load('file://'+SparkFiles.get("summary.parquet"))

summary_df.sort("Country", "WeekNumber").show()

rank_window = Window.partitionBy("Country") \
        .orderBy(f.col("InvoiceValue").desc()) \
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df = summary_df.withColumn("Rank", f.dense_rank().over(rank_window)) \
    .where(f.col("Rank") <= 2) \
    .sort("Country", "Rank") \
    .show()

+---------------+----------+-----------+-------------+------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|
+---------------+----------+-----------+-------------+------------+
|      Australia|        48|          1|          107|      358.25|
|      Australia|        49|          1|          214|       258.9|
|      Australia|        50|          2|          133|      387.95|
|        Austria|        50|          2|            3|      257.04|
|        Bahrain|        51|          1|           54|      205.74|
|        Belgium|        48|          1|          528|       346.1|
|        Belgium|        50|          2|          285|      625.16|
|        Belgium|        51|          2|          942|      838.65|
|Channel Islands|        49|          1|           80|      363.53|
|         Cyprus|        50|          1|          917|     1590.82|
|        Denmark|        49|          1|          454|      1281.5|
|           EIRE|        48|          7|        

**Joins**

In [34]:
    orders_list = [("01", "02", 350, 1),
                   ("01", "04", 580, 1),
                   ("01", "07", 320, 2),
                   ("02", "03", 450, 1),
                   ("02", "06", 220, 1),
                   ("03", "01", 195, 1),
                   ("04", "09", 270, 3),
                   ("04", "08", 410, 2),
                   ("05", "02", 350, 1)]

    order_df = spark.createDataFrame(orders_list).toDF("order_id", "prod_id", "unit_price", "qty")

    product_list = [("01", "Scroll Mouse", 250, 20),
                    ("02", "Optical Mouse", 350, 20),
                    ("03", "Wireless Mouse", 450, 50),
                    ("04", "Wireless Keyboard", 580, 50),
                    ("05", "Standard Keyboard", 360, 10),
                    ("06", "16 GB Flash Storage", 240, 100),
                    ("07", "32 GB Flash Storage", 320, 50),
                    ("08", "64 GB Flash Storage", 430, 25)]

    product_df = spark.createDataFrame(product_list).toDF("prod_id", "prod_name", "list_price", "qty")

    product_df.show()
    order_df.show()

    join_expr = order_df.prod_id == product_df.prod_id

    product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

    order_df.join(product_renamed_df, join_expr, "inner") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id", "prod_id", "prod_name", "unit_price", "list_price", "qty") \
        .show()

+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+

+--------+------

In [35]:
    orders_list = [("01", "02", 350, 1),
                   ("01", "04", 580, 1),
                   ("01", "07", 320, 2),
                   ("02", "03", 450, 1),
                   ("02", "06", 220, 1),
                   ("03", "01", 195, 1),
                   ("04", "09", 270, 3),
                   ("04", "08", 410, 2),
                   ("05", "02", 350, 1)]

    order_df = spark.createDataFrame(orders_list).toDF("order_id", "prod_id", "unit_price", "qty")

    product_list = [("01", "Scroll Mouse", 250, 20),
                    ("02", "Optical Mouse", 350, 20),
                    ("03", "Wireless Mouse", 450, 50),
                    ("04", "Wireless Keyboard", 580, 50),
                    ("05", "Standard Keyboard", 360, 10),
                    ("06", "16 GB Flash Storage", 240, 100),
                    ("07", "32 GB Flash Storage", 320, 50),
                    ("08", "64 GB Flash Storage", 430, 25)]

    product_df = spark.createDataFrame(product_list).toDF("prod_id", "prod_name", "list_price", "qty")

    product_df.show()
    order_df.show()

    join_expr = order_df.prod_id == product_df.prod_id

    product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

    order_df.join(product_renamed_df, join_expr, "left") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id", "prod_id", "prod_name", "unit_price", "list_price", "qty") \
        .withColumn("prod_name", expr("coalesce(prod_name, prod_id)")) \
        .withColumn("list_price", expr("coalesce(list_price, unit_price)")) \
        .sort("order_id") \
        .show()

+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+

+--------+------

**Shuffle Join**

In [36]:
!mkdir data data/d1 data/d2
!wget -O data/d1/part-00000-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/d1/part-00000-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json > /dev/null 2>&1
!wget -O data/d1/part-00001-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/d1/part-00001-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json > /dev/null 2>&1
!wget -O data/d1/part-00002-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/d1/part-00002-00af64b6-7ef5-4909-8f82-b8897114efaf-c000.json > /dev/null 2>&1 

!wget -O data/d2/part-00000-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/Spark-Programming-In-Python-master/19-ShuffleJoinDemo/data/d2/part-00000-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json > /dev/null 2>&1
!wget -O data/d2/part-00001-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/d1/part-00001-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json > /dev/null 2>&1
!wget -O data/d2/part-00002-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/d1/part-00002-ee8814fd-bbb7-4a16-912b-f74b04a6d4fa-c000.json > /dev/null 2>&1

In [40]:
flight_time_df1 = spark.read.json("data/d1/")
flight_time_df2 = spark.read.json("data/d2/")

spark.conf.set("spark.sql.shuffle.partitions", 3)

join_expr = flight_time_df1.id == flight_time_df2.id
join_df = flight_time_df1.join(flight_time_df2, join_expr, "inner")
join_df.explain()

== Physical Plan ==
*(5) SortMergeJoin [id#2812L], [id#2836L], Inner
:- *(2) Sort [id#2812L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(id#2812L, 3), ENSURE_REQUIREMENTS, [id=#1185]
:     +- *(1) Filter isnotnull(id#2812L)
:        +- FileScan json [DEST#2805,DEST_CITY_NAME#2806,FL_DATE#2807,OP_CARRIER#2808,OP_CARRIER_FL_NUM#2809L,ORIGIN#2810,ORIGIN_CITY_NAME#2811,id#2812L] Batched: false, DataFilters: [isnotnull(id#2812L)], Format: JSON, Location: InMemoryFileIndex[file:/content/data/d1], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<DEST:string,DEST_CITY_NAME:string,FL_DATE:string,OP_CARRIER:string,OP_CARRIER_FL_NUM:bigin...
+- *(4) Sort [id#2836L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#2836L, 3), ENSURE_REQUIREMENTS, [id=#1193]
      +- *(3) Filter isnotnull(id#2836L)
         +- FileScan json [ARR_TIME#2828L,CANCELLED#2829L,CRS_ARR_TIME#2830L,CRS_DEP_TIME#2831L,DEP_TIME#2832L,DISTANCE#2833L,TAXI_IN#2834L,WHEELS_ON#2835

**View Log**

In [ ]:
!cat app-logs/sparklog.log